## Анализ и визуализация данных на языке Python

### Библиотека pandas.


- "Широкий" и "Длинный" форматы таблиц (stack/unstack)
- Механизм `pivot_tables`



In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display

### "Раскатка" и "Штабелирование" данных

Работа со сложным индексм: функции ```stack()``` и ```unstack()```.

In [2]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


 Функция ```stack()``` "сложит" данные вертикально, построив "сложный индекс":

In [3]:
data
stacked = data.stack()
stacked.to_frame(name='values')

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


values
state    number        
Ohio     one          0
         two          1
         three        2
Colorado one          3
         two          4
         three        5

In [4]:
stacked.index

MultiIndex([(    'Ohio',   'one'),
            (    'Ohio',   'two'),
            (    'Ohio', 'three'),
            ('Colorado',   'one'),
            ('Colorado',   'two'),
            ('Colorado', 'three')],
           names=['state', 'number'])

Функция ```unstack()``` выполнит противоположную задачу - данные из "сложного индекса" вынесет в колонки:

In [5]:
stacked
stacked.unstack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


#### Функции melt() и pivot()

```melt()``` - преобразует столбцы в строки, добавляя соотвествующие столбцы variable и value. \
```pivot()```- наоборот, собирает данные по строкам в столбцы

In [6]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [7]:
df
melted = pd.melt(df, ['key'], var_name='letters', value_name='numbers')
melted

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


,key,letters,numbers
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [8]:
melted
reshaped = pd.pivot(melted, index=['key'], columns=['letters'], values='numbers')
reshaped

,key,letters,numbers
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


letters,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


### Функции pivot_table()

In [9]:
df_titanic = pd.read_csv('data/titanic.csv', index_col='PassengerId')

In [10]:
df_titanic.groupby('Sex')[['Survived']].mean()

,Survived
Sex,
female,0.742038
male,0.188908


Сразу же можно сделать вывод о том, что из каждых четырех женщин, находившихся на борту, выжили три, в то время как из каждых пяти мужчин выжил только один!

Это интересная информация, но мы можем пойти дальше и выяснить взаимосвязь между показателем выживаемости и двумя другими параметрами, такими как пол и, например, класс.

In [11]:
df_titanic.groupby(['Sex', 'Pclass'])['Survived'].aggregate('mean').to_frame()

Survived
Sex    Pclass          
female 1       0.968085
       2       0.921053
       3       0.500000
male   1       0.368852
       2       0.157407
       3       0.135447

In [12]:
df_titanic.groupby(['Sex', 'Pclass'])['Survived'].aggregate('mean').unstack()

Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Теперь мы имеем четкое представление о том, как пол и класс повлияли на выживаемость, но код становится немного громоздким.   Хотя каждый шаг этой последовательности вполне понятен в свете рассмотренных выше инструментов, тем не менее длинную строку кода достаточно трудно читать и использовать. Подобные операции широко распространены, в связи с чем библиотека Pandas имеет в своем составе специальный метод `pivot_table`, лаконично реализующий данный тип многомерной агрегации.

Ниже представлен эквивалент рассмотренной выше операции, реализованный с помощью метода `pivot_table` объекта DataFrame:

In [13]:
df_titanic.pivot_table('Survived', index='Sex', columns='Pclass')

Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Группирование в сводной таблице может иметь несколько уровней и задаваться посредством различных параметров. Например, в качестве третьего измерения нас может заинтересовать возраст.   
Мы разделим возраст на интервалы, с помощью функции `pd.cut`:

In [14]:
age = pd.cut(df_titanic['Age'], bins=[0, 18, 80], labels=['young', "adult"])
df_titanic.pivot_table('Survived', ['Sex', age], 'Pclass', observed=False)

Pclass               1         2         3
Sex    Age                                
female young  0.909091  1.000000  0.511628
       adult  0.972973  0.900000  0.423729
male   young  0.800000  0.600000  0.215686
       adult  0.375000  0.071429  0.133663

#### Дополнительные параметры сводной таблицы

Полная сигнатура вызова метода `pivot_table` объекта DataFrame является следующей:  
`DataFrame.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True)`



* `DataFrame` &mdash; исходная таблица;
* `values` &mdash; аггрeгируемый столбец, его значения непосредственно определяют значения сводной таблицы;
* `index` &mdash; ключи для группировки, относятся к индексам сводной таблицы;
* `columns` &mdash; ключи для группировки, относятся к столбцам сводной таблицы;
* `aggfunc` &mdash; функция, которая будет применена к каждой группе значений `values`, сгруппированным по значениям `index` и `columns`. Значения этой функции и есть значения сводной таблицы. Если передается список функций, то сводная таблица имеет иерархические имена колонок, верхние значения которых &mdash; имена функций;
* `fill_value` &mdash; значения для замены пропусков;
* `dropna` &mdash; не включать столбцы, которые состоят только из `NaN`;
* `margins` &mdash; добавляет результирующий столбец/строку;



Выше мы рассмотрели три первых параметра. Теперь давайте обсудим остальные. Параметры `fill_value` и `dropna` задают способ обработки отсутствующих данных. Их использование не вызывает затруднений, поэтому мы не будем приводить примеры.

Параметр `aggfunc`задает тип агрегации. По умолчанию его значение равно `mean`. Как и в случае GroupBy, тип агрегации можно задать либо с помощью предопределенной строки (например, `sum`, `mean`, `count`, `min`, `max` и др.), либо посредством функции, реализующей агрегацию (например, np.sum(), min(), sum() и др.).   
Кроме того, этот параметр может быть задан в виде словаря, отображающего столбцы на любые из желаемых значений, перечисленных выше:

In [15]:
df_titanic.pivot_table(
    index='Sex',
    columns='Pclass',
    aggfunc={'Survived':"sum", 'Fare':'mean'}
)

Fare                       Survived        
Pclass           1          2          3        1   2   3
Sex                                                      
female  106.125798  21.970121  16.118810       91  70  72
male     67.226127  19.741782  12.661633       45  17  47

Обратите внимание, в данном случае мы не задали параметр `values`, потому что он задается автоматически, когда параметр `aggfunc` представлен в виде словаря.  

Иногда требуется вычислить обобщенные значения по каждой группе. Это можно сделать с помощью параметра `margins`:

In [16]:
df_titanic.pivot_table('Survived', index='Sex', columns='Pclass', margins=True)

Pclass,1,2,3,All
Sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


Представленный выше код автоматически дает нам процент выживших в зависимости от пола без учета класса, в зависимости от класса без учета пола, а также общий процент выживших, составляющий __38%__.